In [1]:
!pip install -q trino

In [13]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'lakekeeper' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://lakekeeper:8181/catalog"
TRINO_URI = "http://trino:8080"
WAREHOUSE = "irisa-ot"

# Create Trino Catalog

In [14]:
from trino.dbapi import connect

conn = connect(host=TRINO_URI, user="trino")

In [16]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'NONE',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
"""
)

TrinoUserError: TrinoUserError(type=USER_ERROR, name=ALREADY_EXISTS, message="Catalog 'lakekeeper' already exists", query_id=20250722_093827_00001_2zwd7)

## Read and Write Tables

In [5]:
# Connect directly to demo catalog, so that we don't have to use it as a prefix
conn = connect(host=TRINO_URI, user="trino", catalog="lakekeeper")
cur = conn.cursor()

In [9]:
cur.execute("CREATE SCHEMA IF NOT EXISTS trino_namespace")

In [11]:
cur.execute(
    "CREATE TABLE IF NOT EXISTS trino_namespace.trino_table (my_ints INT, my_floats DOUBLE, strings VARCHAR) WITH (format='PARQUET')"
)
cur.execute(
    "INSERT INTO trino_namespace.trino_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)

In [12]:
cur.execute("SELECT * FROM trino_namespace.trino_table").fetchall()

[[1, 1.0, 'a'], [1, 1.0, 'a'], [2, 2.0, 'b'], [2, 2.0, 'b']]